In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
CHP_ID = "464"
SUBMODEL = "cond-detr-50" # "cond-detr-50"
MODEL_PATH = f"logs/model2/checkpoint-{CHP_ID}"
# IMAGE_SHAPE = 1333 _Shape{IMAGE_SHAPE}
THR = 0.0
iou_threshold = 0.4
FILE_NAME = f"{SUBMODEL}_THR{THR*100:.3f}_IOU{iou_threshold:.3f}_ID{CHP_ID}"

In [3]:
from typing import List, Tuple, Union

import numpy as np
import pandas as pd
import torch
from PIL import Image
from torchvision.ops import nms
from tqdm import tqdm
from transformers import (AutoImageProcessor, AutoModelForObjectDetection,
                          ConditionalDetrForObjectDetection,
                          ConditionalDetrImageProcessor)
from transformers.image_transforms import center_to_corners_format
from transformers.utils.generic import TensorType

from zindi_code import CLS_MAPPER
from zindi_code.dataset_class import load_and_format

In [4]:

image_folder = "zindi_data/images"

test = load_and_format("zindi_data/ValDataset.csv")
test.sample(5)

,image_id,bbox,category_id,id
717,id_qa70t0m4yc.jpg,"[2515, 1242, 2596, 1313]",0,717
172,id_0fdars2kkw.jpg,"[586, 846, 620, 878]",0,172
874,id_dtffc5i3ry.jpg,"[673, 607, 724, 656]",1,874
423,id_wy8p0eu51i.jpg,"[392, 550, 447, 612]",1,423
64,id_ctkvvkc84k.jpg,"[881, 840, 942, 906]",1,64


In [5]:
model_pth = MODEL_PATH

image_processor: ConditionalDetrImageProcessor = AutoImageProcessor.from_pretrained(
    model_pth, use_fast=False,
)
model: ConditionalDetrForObjectDetection = AutoModelForObjectDetection.from_pretrained(
    model_pth
)

model = model.eval().to(
    "cuda"
) # .train(False)

In [6]:
image_processor

ConditionalDetrImageProcessor {
  "do_convert_annotations": true,
  "do_normalize": true,
  "do_pad": true,
  "do_rescale": true,
  "do_resize": true,
  "format": "coco_detection",
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "ConditionalDetrImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "pad_size": {
    "height": 800,
    "width": 1333
  },
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "max_height": 800,
    "max_width": 1333
  }
}

In [7]:
model.config.id2label

{0: 'Trophozoite', 1: 'WBC'}

In [8]:
def make_prediction(image: Image.Image):
    inputs = image_processor(images=[image], return_tensors="pt")
    outputs = model(**inputs.to("cuda"))
    target_sizes = torch.tensor([[image.size[1], image.size[0]]])
    results = image_processor.post_process_object_detection(
        outputs, threshold=THR, target_sizes=target_sizes
    )[0]
    return results


def _make_predictions(images: list[Image.Image]):
    inputs = image_processor(
        images=[np.array(image) for image in images], return_tensors="pt"
    ).to("cuda")
    # inputs = image_processor.pad(inputs)
    outputs = model(**inputs)
    target_sizes = torch.tensor([image.size[::-1] for image in images])
    return image_processor.post_process_object_detection(
        outputs, threshold=THR, target_sizes=target_sizes
    )


@torch.no_grad()
def make_predictions(images: list[Image.Image]):
    predictions = [make_prediction(image) for image in images]
    return predictions


def load_transform(path: str):
    return Image.open(os.path.join(image_folder, path)).convert("RGB")


def load_images(image_pths: list[str]):
    return [load_transform(image_pth) for image_pth in image_pths]


def predicts(image_pths: list[str]):
    images = load_images(image_pths)
    results = make_predictions(images)
    predictions = []
    for image_pth, result in zip(image_pths, results):
        prediction = []
        if len(result["boxes"]):
            indices = nms(result["boxes"], result["scores"], iou_threshold)
            if not len(indices):
                continue
            for score, label, box in zip(
                result["scores"][indices],
                result["labels"][indices],
                result["boxes"][indices],
            ):
                x1, y1, x2, y2 = (round(i, 2) for i in box.tolist())
                prediction.append(
                    [
                        image_pth,
                        x1,
                        y1,
                        x2 - x1,
                        y2 - y1,
                        model.config.id2label[label.item()],
                        round(score.item(), 3),
                    ]
                )
        if not len(prediction):
            prediction.append([image_pth, 0, 0, 0, 0, "NEG", 1.0])
        predictions.extend(prediction)
    return pd.DataFrame(
        predictions, columns=["image_id", "x", "y", "w", "h", "category_id", "score"]
    )

In [9]:
image_pths = test["image_id"].unique()[:16]
image_pths

array(['id_w8xnbd5rvm.jpg', 'id_ytq3slqkjm.jpg', 'id_e20xnaq5qn.jpg',
       'id_7fc9zyfy0e.jpg', 'id_6g52lmvz2y.jpg', 'id_z0i61ad0tq.jpg',
       'id_55a6sf8hbe.jpg', 'id_dg0icorzno.jpg', 'id_zdg96srigj.jpg',
       'id_ezd6x40fd0.jpg', 'id_ch6r0g46fr.jpg', 'id_4cotsn0obm.jpg',
       'id_0fdars2kkw.jpg', 'id_4wkzpeu6or.jpg', 'id_idjqlz4ppb.jpg',
       'id_by6e6shi2z.jpg'], dtype=object)

In [10]:
results = predicts(image_pths)

In [11]:
results["category_id"].value_counts()

category_id
Trophozoite    654
WBC             94
Name: count, dtype: int64

In [12]:
results.sample(10)

,image_id,x,y,w,h,category_id,score
183,id_7fc9zyfy0e.jpg,2434.29,920.61,93.13,119.26,Trophozoite,0.243
337,id_dg0icorzno.jpg,560.36,736.58,30.19,28.37,Trophozoite,0.258
87,id_ytq3slqkjm.jpg,1294.39,277.03,32.35,31.83,Trophozoite,0.340
496,id_ch6r0g46fr.jpg,921.96,1283.94,74.51,92.50,Trophozoite,0.271
524,id_ch6r0g46fr.jpg,2596.14,1984.62,82.66,102.40,Trophozoite,0.177
97,id_ytq3slqkjm.jpg,610.21,895.81,32.43,31.05,Trophozoite,0.317
737,id_by6e6shi2z.jpg,670.61,424.23,31.86,29.12,Trophozoite,0.274
733,id_by6e6shi2z.jpg,1518.25,14.19,36.67,32.52,Trophozoite,0.281
435,id_ezd6x40fd0.jpg,953.66,95.77,29.84,30.36,Trophozoite,0.339
730,id_by6e6shi2z.jpg,281.32,380.97,35.75,32.89,Trophozoite,0.290


In [13]:
batch_size = 16
test_images = test["image_id"].unique()
results = [
	predicts(test_images[i : i + batch_size])
	for i in tqdm(
		range(0, len(test_images), batch_size), total=len(test_images) // batch_size + 1
	) if i < len(test_images)
]

100%|██████████| 18/18 [01:21<00:00,  4.55s/it]


In [14]:
predictions = pd.concat(results, ignore_index=True)

In [15]:
predictions.sample(10)

,image_id,x,y,w,h,category_id,score
8754,id_yinirldwcz.jpg,585.17,1614.98,89.65,110.49,Trophozoite,0.339
4676,id_0nm5i3vtg9.jpg,942.58,699.92,28.72,28.96,Trophozoite,0.300
7295,id_ebv1caqkrd.jpg,889.37,569.34,29.02,29.16,Trophozoite,0.280
5218,id_56cm3hgim3.jpg,807.76,2553.06,90.03,118.08,Trophozoite,0.170
547,id_4cotsn0obm.jpg,1271.22,604.45,33.16,32.78,Trophozoite,0.279
9006,id_aviegh5t3l.jpg,2360.99,908.14,109.80,145.26,WBC,0.264
7225,id_igebahf2c3.jpg,1931.81,1969.17,106.52,134.02,Trophozoite,0.077
10348,id_ynfnpmojzt.jpg,2862.34,1393.12,149.06,176.55,WBC,0.361
7160,id_55n7okb0sj.jpg,1588.17,521.49,33.23,37.45,Trophozoite,0.093
7512,id_5283833d2l.jpg,2198.76,461.47,91.93,114.60,Trophozoite,0.180


In [16]:
predictions["category_id"].value_counts(True)

category_id
Trophozoite    0.825375
WBC            0.174625
Name: proportion, dtype: float64

In [17]:
predictions["score"].describe()

count    11476.000000
mean         0.244534
std          0.111838
min          0.055000
25%          0.156000
50%          0.236000
75%          0.313000
max          0.664000
Name: score, dtype: float64

In [18]:
f"zindi_data/validation/prediction_{FILE_NAME}.csv"

'zindi_data/validation/prediction_cond-detr-50_THR0.000_IOU0.400_ID464.csv'

In [19]:
predictions = predictions.rename(columns={"x": "xmin", "y": "ymin"})
predictions = predictions.rename(columns={"category_id": "class", "image_id": "Image_ID", "score": "confidence"})

predictions["xmax"] = predictions["xmin"] + predictions["w"]
predictions["ymax"] = predictions["ymin"] + predictions["h"]

predictions.to_csv(f"zindi_data/validation/prediction_{FILE_NAME}.csv", index=False)